## Create Example Jobs

Create various types of example jobs:

In [1]:
from scm.plams import from_smiles, AMSJob, PlamsError, Settings, Molecule, Atom
from scm.libbase import UnifiedChemicalSystem as ChemicalSystem
from scm.input_classes.drivers import AMS
from scm.input_classes.engines import DFTB
from scm.utils.conversions import plams_molecule_to_chemsys

def example_job_dftb(smiles, task, use_chemsys = False):
    # Generate molecule from smiles
    mol = from_smiles(smiles)
    if use_chemsys:
        mol = plams_molecule_to_chemsys(mol)

    # Set up calculation settings using PISA
    sett = Settings()
    sett.runscript.nproc = 1
    driver = AMS()
    driver.Task = task
    driver.Engine = DFTB() 
    sett.input = driver
    return AMSJob(molecule = mol, settings = sett)


def example_job_adf(smiles, task, basis, gga = None, use_chemsys = False):
    # Generate molecule from smiles
    mol = from_smiles(smiles)
    if use_chemsys:
        mol = plams_molecule_to_chemsys(mol)

    # Set up calculation settings using standard settings
    sett = Settings()
    sett.runscript.nproc = 1
    sett.input.AMS.Task = task
    sett.input.ADF.Basis.Type = basis
    if gga:
        sett.input.ADF.XC.GGA = gga
    return AMSJob(molecule = mol, settings = sett)
    

def example_job_neb(iterations, use_chemsys = False):
    # Set up molecules
    main_molecule = Molecule()
    main_molecule.add_atom(Atom(symbol="C", coords=(0, 0, 0)))
    main_molecule.add_atom(Atom(symbol="N", coords=(1.18, 0, 0)))
    main_molecule.add_atom(Atom(symbol="H", coords=(2.196, 0, 0)))
    final_molecule = main_molecule.copy()
    final_molecule.atoms[1].x = 1.163
    final_molecule.atoms[2].x = -1.078

    mol = {"": main_molecule, "final": final_molecule}

    if use_chemsys:
        mol = {k: plams_molecule_to_chemsys(v) for k, v in mol.items()}

    # Set up calculation settings
    sett = Settings()
    sett.runscript.nproc = 1
    sett.input.ams.Task = "NEB"
    sett.input.ams.NEB.Images = 9
    sett.input.ams.NEB.Iterations = iterations
    sett.input.DFTB
        
    return AMSJob(molecule = mol, settings = sett)

Run example jobs:

In [2]:
from scm.plams import config, JobRunner
config.default_jobrunner = JobRunner(parallel=True, maxthreads = 8)

smiles = ["CC", "C", "O", "CO"]
tasks = ["SinglePoint", "GeometryOptimization"]
engines = ["DFTB", "ADF"]
jobs = []
for i, s in enumerate(smiles):
    for j, t in enumerate(tasks):
        job_dftb = example_job_dftb(s, t, use_chemsys = i % 2)
        job_adf1 = example_job_adf(s, t, "DZ", use_chemsys = True)
        job_adf2 = example_job_adf(s, t, "TZP", "PBE")
        jobs += [job_dftb, job_adf1, job_adf2]

job_neb1 = example_job_neb(10)
job_neb2 = example_job_neb(100, use_chemsys=True)
jobs += [job_neb1, job_neb2]

for j in jobs:
    j.run()

[23.01|09:33:00] JOB plamsjob STARTED
[23.01|09:33:00] JOB plamsjob STARTED
[23.01|09:33:00] JOB plamsjob STARTED
[23.01|09:33:00] Renaming job plamsjob to plamsjob.002
[23.01|09:33:00] Renaming job plamsjob to plamsjob.003
[23.01|09:33:00] JOB plamsjob STARTED
[23.01|09:33:00] JOB plamsjob STARTED
[23.01|09:33:00] JOB plamsjob RUNNING
[23.01|09:33:00] JOB plamsjob.002 RUNNING
[23.01|09:33:00] JOB plamsjob STARTED
[23.01|09:33:00] JOB plamsjob STARTED
[23.01|09:33:00] JOB plamsjob STARTED
[23.01|09:33:00] Renaming job plamsjob to plamsjob.004
[23.01|09:33:00] JOB plamsjob STARTED
[23.01|09:33:00] JOB plamsjob.003 RUNNING
[23.01|09:33:00] Renaming job plamsjob to plamsjob.005
[23.01|09:33:00] Renaming job plamsjob to plamsjob.006
[23.01|09:33:00] Renaming job plamsjob to plamsjob.007
[23.01|09:33:00] Renaming job plamsjob to plamsjob.008
[23.01|09:33:00] JOB plamsjob.005 RUNNING
[23.01|09:33:00] JOB plamsjob.004 RUNNING
[23.01|09:33:00] JOB plamsjob.006 RUNNING
[23.01|09:33:00] JOB plam

In [3]:
for j in jobs:
    j.results.wait()

[23.01|09:33:20] Waiting for job plamsjob.004 to finish
[23.01|09:33:37] Waiting for job plamsjob.006 to finish


Delete the .dill file associated with some jobs for testing:

In [4]:
from pathlib import Path
import os 

for j in jobs:
    if j.name.endswith(".005"):
        path = Path(j.path) / f"{j.name}.dill"
        os.remove(path)

## Job Analysis

### Loading Jobs

Jobs can be loaded by passing job objects, or loading from a path.

In [5]:
from scm.plams import JobAnalysis

In [6]:
ja = JobAnalysis(jobs = jobs[:4], paths = [j.path for j in jobs[4:]])

By default there are basic job information fields which are are set up.

In [7]:
print(ja.to_table(max_col_width=40))

| Path                                        | Name         | OK   | Check | ErrorMsg                          |
|---------------------------------------------|--------------|------|-------|-----------------------------------|
| /Users/ormrodmorley/Documents/code/ams/a... | plamsjob     | True | True  | None                              |
| /Users/ormrodmorley/Documents/code/ams/a... | plamsjob.003 | True | True  | None                              |
| /Users/ormrodmorley/Documents/code/ams/a... | plamsjob.002 | True | True  | None                              |
| /Users/ormrodmorley/Documents/code/ams/a... | plamsjob.005 | True | True  | None                              |
| /Users/ormrodmorley/Documents/code/ams/a... | plamsjob.004 | True | True  | None                              |
| /Users/ormrodmorley/Documents/code/ams/a... | plamsjob.006 | True | True  | None                              |
| /Users/ormrodmorley/Documents/code/ams/a... | plamsjob.007 | True | True  | None      

### Adding, Removing and Modifying Fields

Other standard and also custom fields can be added and removed.

In [8]:
ja.add_molecule_fields()
ja.add_timing_fields()
ja.add_settings_input_fields()

ja.add_field("Energy", lambda j: j.results.get_energy())
ja["NAtoms"] = lambda j: len(j.molecule)  # alternative syntax for adding fields via dict notation
ja.NSpin = lambda j: j.results.get_n_spin() # alternative syntax for adding fields via dot notation

ja.remove_field("ElapsedTime") 
del ja["Path"]  # alternative syntax for removing fields via dict notation
del ja.SysTime  # alternative syntax for removing fields via dot notation

ja.rename_field("InputAmsTask", "Task")
ja.rename_field("InputAdfBasisType", "Basis")

In [9]:
print(ja.to_table(max_col_width = 30))

| Name         | OK   | Check | ErrorMsg                          | Formula           | Smiles            | CPUTime            | Task                 | Basis | InputAdfXcGga | InputAmsNebImages | InputAmsNebIterations | Energy                            | NAtoms | NSpin                             |
|--------------|------|-------|-----------------------------------|-------------------|-------------------|--------------------|----------------------|-------|---------------|-------------------|-----------------------|-----------------------------------|--------|-----------------------------------|
| plamsjob     | True | True  | None                              | C2H6              | CC                | 0.201668           | SinglePoint          | None  | None          | None              | None                  | -7.462962534848951                | 8      | 1                                 |
| plamsjob.003 | True | True  | None                              | C2H6              | CC       

Analysis fields and jobs can be filtered out based on values.

In [10]:
ja.filter_jobs(lambda data: data["ErrorMsg"] is not None)
ja.filter_fields(lambda vals: any([isinstance(v, float) and v > 10 for v in vals]))

ja.remove_uniform_fields(ignore_empty = True)
ja.remove_empty_fields()

In [11]:
print(ja.to_table(max_col_width = 30))

| Name         | Formula           | Smiles            | Task                 | Basis | Energy              | NAtoms |
|--------------|-------------------|-------------------|----------------------|-------|---------------------|--------|
| plamsjob     | C2H6              | CC                | SinglePoint          | None  | -7.462962534848951  | 8      |
| plamsjob.003 | C2H6              | CC                | SinglePoint          | DZ    | -1.51334657536796   | 8      |
| plamsjob.002 | C2H6              | CC                | SinglePoint          | TZP   | -1.477488924954076  | 8      |
| plamsjob.005 | C2H6              | CC                | GeometryOptimization | None  | -7.468180807890643  | 8      |
| plamsjob.004 | C2H6              | CC                | GeometryOptimization | DZ    | -1.5188342118440623 | 8      |
| plamsjob.006 | C2H6              | CC                | GeometryOptimization | TZP   | -1.4824247864004538 | 8      |
| plamsjob.007 | CH4               | C          

### Extracting Analysis Data

Analysis data can be extracted in a variety of ways. The data can be obtained in a markdown compatible table:

In [12]:
ja.filter_jobs(lambda data: data["Task"] != "GeometryOptimization" or data["Basis"] is None)
print(ja.to_table(max_col_width = 30, max_rows = 5))

| Name         | Formula | Smiles | Task                 | Basis | Energy              | NAtoms |
|--------------|---------|--------|----------------------|-------|---------------------|--------|
| plamsjob.004 | C2H6    | CC     | GeometryOptimization | DZ    | -1.5188342118440623 | 8      |
| plamsjob.006 | C2H6    | CC     | GeometryOptimization | TZP   | -1.4824247864004538 | 8      |
| ...          | ...     | ...    | ...                  | ...   | ...                 | ...    |
| plamsjob.018 | H2O     | O      | GeometryOptimization | TZP   | -0.5194311491758227 | 3      |
| plamsjob.023 | CH4O    | CO     | GeometryOptimization | DZ    | -1.1062334880613056 | 6      |
| plamsjob.024 | CH4O    | CO     | GeometryOptimization | TZP   | -1.1046970013081945 | 6      |


The data can be accessed via the `get_analysis` method, which returns a dictionary of all values.

In [13]:
analysis = ja.get_analysis()
for smiles, basis, energy in zip(analysis["Smiles"] , analysis["Basis"], analysis["Energy"]):
    print(f"{smiles} ({basis}): {energy}")

CC (DZ): -1.5188342118440623
CC (TZP): -1.4824247864004538
C (DZ): -0.895063427538391
C (TZP): -0.880063379024256
O (DZ): -0.501352093695396
O (TZP): -0.5194311491758227
CO (DZ): -1.1062334880613056
CO (TZP): -1.1046970013081945


Data for specific fields can also be accessed via item / attribute getters.

In [14]:
for smiles, n in zip(ja["Smiles"], ja.NAtoms):
    print(f"{smiles}: {n}")

CC: 8
CC: 8
C: 5
C: 5
O: 3
O: 3
CO: 6
CO: 6


Data can also be written to a csv file.

In [15]:
from pathlib import Path
import os

path = Path(".") / "analysis.csv"
ja.to_csv_file(path)

with open(path) as f:
    print(f.read())

os.remove(path);

Name,Formula,Smiles,Task,Basis,Energy,NAtoms
plamsjob.004,C2H6,CC,GeometryOptimization,DZ,-1.5188342118440623,8
plamsjob.006,C2H6,CC,GeometryOptimization,TZP,-1.4824247864004538,8
plamsjob.011,CH4,C,GeometryOptimization,DZ,-0.895063427538391,5
plamsjob.012,CH4,C,GeometryOptimization,TZP,-0.880063379024256,5
plamsjob.017,H2O,O,GeometryOptimization,DZ,-0.501352093695396,3
plamsjob.018,H2O,O,GeometryOptimization,TZP,-0.5194311491758227,3
plamsjob.023,CH4O,CO,GeometryOptimization,DZ,-1.1062334880613056,6
plamsjob.024,CH4O,CO,GeometryOptimization,TZP,-1.1046970013081945,6



Of finally exported to a pandas dataframe.

In [16]:
ja.to_dataframe()

,Name,Formula,Smiles,Task,Basis,Energy,NAtoms
0,plamsjob.004,C2H6,CC,GeometryOptimization,DZ,-1.518834,8
1,plamsjob.006,C2H6,CC,GeometryOptimization,TZP,-1.482425,8
2,plamsjob.011,CH4,C,GeometryOptimization,DZ,-0.895063,5
3,plamsjob.012,CH4,C,GeometryOptimization,TZP,-0.880063,5
4,plamsjob.017,H2O,O,GeometryOptimization,DZ,-0.501352,3
5,plamsjob.018,H2O,O,GeometryOptimization,TZP,-0.519431,3
6,plamsjob.023,CH4O,CO,GeometryOptimization,DZ,-1.106233,6
7,plamsjob.024,CH4O,CO,GeometryOptimization,TZP,-1.104697,6


This may be the most powerful and useful option, as then pandas can be used for the data manipulation.